<a href="https://colab.research.google.com/github/YinGuoX/Deep_Learning_Keras_WithDeeplizard/blob/master/7_Neural_Network_Predictions_With_TensorFlow's_Keras_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Neural Network Predictions With TensorFlow's Keras API

在本集中，我们将演示如何使用神经网络进行推理以对测试集中的数据进行预测。 我们将继续使用与前几集相同的tf.keras.Sequential模型和数据。

正如我们之前提到的那样，当我们训练模型时，希望我们以后能够采用训练后的模型，将其应用于新数据，并使模型进行概括并准确地预测之前从未见过的数据 。

例如，假设我们有一个对猫或狗的图像进行分类的模型，并且训练数据包含来自特定在线数据集的数千个猫和狗的图像。

## 1. 什么是推理

现在假设以后我们要采用该模型并将其用于从不同数据集中预测猫和狗的其他图像。 希望是，即使我们的模型在训练过程中没有训练过这些特定的猫狗图像中，它仍然能够基于从猫和狗数据集中学到的信息为它们准确地做出预测。

我们称这个过程为推理，因为模型使用它从训练中获得的知识，并使用它来推断预测或结果。

至此，我们在过去几集中一直在使用的模型已经过培训和验证。 鉴于我们从验证数据中看到的结果，该模型似乎可以很好地预测新的测试集。

注意，测试集是训练结束后专门用于推理的数据集。 您可以在“深度学习基础知识”课程中找到有关深度学习中使用的所有不同类型数据集的更多信息。

## 2. 创建测试集
我们将以创建训练集的相同方式创建测试集。 通常，应始终以与训练集相同的方式处理测试集。

我们不会逐步介绍下面用于生成和处理测试数据的代码，因为在生成训练数据的前一集中已经详细介绍了该代码，但是请确保您已将所有导入内容都包含在其中。 从之前的情节开始到现在为止，以及所有现有的代码。

In [ ]:
# 准备数据
import numpy as np
from random import randint
from sklearn.utils import shuffle
from sklearn.preprocessing import MinMaxScaler

train_labels = []
train_samples = []

# 生成数据
for i in range(50):
    # 大约5%的年轻人确实经历过副作用
    random_younger = randint(13,64)
    train_samples.append(random_younger)
    train_labels.append(1)

    # 大约5%的老年人没有经历过副作用
    random_older = randint(65,100)
    train_samples.append(random_older)
    train_labels.append(0)

for i in range(1000):
    # 大约95%的年轻人没有经历过副作用
    random_younger = randint(13,64)
    train_samples.append(random_younger)
    train_labels.append(0)

    # 大约95%的老年人确实经历过副作用
    random_older = randint(65,100)
    train_samples.append(random_older)
    train_labels.append(1)

train_labels = np.array(train_labels)
train_samples = np.array(train_samples)
train_labels,train_samples = shuffle(train_labels,train_samples)
print(train_samples.shape)

# 通过将每个特征缩放到给定的范围来转换特征。
scaler = MinMaxScaler(feature_range=(0,1))

scaled_train_samples = scaler.fit_transform(train_samples.reshape(-1,1))
# 我们只是根据fit_transform（）函数默认情况下不接受一维数据的情况，将数据重塑为2D。
print(train_samples.reshape(-1,1).shape)

(2100,)
(2100, 1)


In [ ]:


test_labels =  []

test_samples = []

for i in range(10):
    # The 5% of younger individuals who did experience side effects
    random_younger = randint(13,64)
    test_samples.append(random_younger)
    test_labels.append(1)

    # The 5% of older individuals who did not experience side effects
    random_older = randint(65,100)
    test_samples.append(random_older)
    test_labels.append(0)

for i in range(200):
    # The 95% of younger individuals who did not experience side effects
    random_younger = randint(13,64)
    test_samples.append(random_younger)
    test_labels.append(0)

    # The 95% of older individuals who did experience side effects
    random_older = randint(65,100)
    test_samples.append(random_older)
    test_labels.append(1)

test_labels = np.array(test_labels)
test_samples = np.array(test_samples)
test_labels, test_samples = shuffle(test_labels, test_samples)

scaled_test_samples = scaler.fit_transform(test_samples.reshape(-1,1))

In [ ]:
# 创建并且编译模型
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy

model = Sequential([
    Dense(units=16, input_shape=(1,), activation='relu'),
    Dense(units=32, activation='relu'),
    Dense(units=2, activation='softmax')
])

model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


In [ ]:
# 训练模型
model.fit(
    x = scaled_train_samples,
    y = train_labels,
    validation_split=0.1,
    batch_size = 10,
    epochs = 30,
    verbose=2
)

Epoch 1/30
189/189 - 1s - loss: 0.7029 - accuracy: 0.4074 - val_loss: 0.6829 - val_accuracy: 0.4714
Epoch 2/30
189/189 - 0s - loss: 0.6623 - accuracy: 0.5593 - val_loss: 0.6581 - val_accuracy: 0.5238
Epoch 3/30
189/189 - 0s - loss: 0.6341 - accuracy: 0.6021 - val_loss: 0.6375 - val_accuracy: 0.5762
Epoch 4/30
189/189 - 0s - loss: 0.6093 - accuracy: 0.6434 - val_loss: 0.6169 - val_accuracy: 0.6238
Epoch 5/30
189/189 - 0s - loss: 0.5854 - accuracy: 0.6725 - val_loss: 0.5965 - val_accuracy: 0.6571
Epoch 6/30
189/189 - 0s - loss: 0.5639 - accuracy: 0.7074 - val_loss: 0.5752 - val_accuracy: 0.7000
Epoch 7/30
189/189 - 0s - loss: 0.5405 - accuracy: 0.7481 - val_loss: 0.5525 - val_accuracy: 0.7286
Epoch 8/30
189/189 - 0s - loss: 0.5175 - accuracy: 0.7778 - val_loss: 0.5303 - val_accuracy: 0.7714
Epoch 9/30
189/189 - 0s - loss: 0.4953 - accuracy: 0.7995 - val_loss: 0.5096 - val_accuracy: 0.7762
Epoch 10/30
189/189 - 0s - loss: 0.4738 - accuracy: 0.8122 - val_loss: 0.4884 - val_accuracy: 0.8429

## 3. 评估测试集
为了从测试集的模型中获得预测，我们调用model.predict（）。

In [ ]:
predictions = model.predict(
    x=scaled_test_samples,
    batch_size = 10,
    verbose=0
)

对于predict()函数，我们传递测试样本x，指定batch_size，并指定在预测生成期间希望从日志消息中获取的详细级别。 由于预测的输出与我们无关，因此我们将verbose = 0设置为无输出。

**注意：**与训练和验证集不同，我们在推断阶段不会将测试集的标签传递给模型。

要查看模型的预测是什么样子的，我们可以遍历它们并打印出来。




In [ ]:
for i in predictions:
  print(i)

[0.04284908 0.957151  ]
[0.09522419 0.9047758 ]
[0.10627808 0.89372194]
[0.91552067 0.08447936]
[0.17964612 0.82035387]
[0.9194009  0.08059911]
[0.21848594 0.7815141 ]
[0.90894824 0.09105177]
[0.01419716 0.98580277]
[0.01743693 0.98256314]
[0.01419716 0.98580277]
[0.01937912 0.9806209 ]
[0.9194009  0.08059911]
[0.06065821 0.93934184]
[0.92383343 0.07616653]
[0.86882836 0.13117167]
[0.24005216 0.75994784]
[0.86882836 0.13117167]
[0.01937912 0.9806209 ]
[0.88110024 0.11889981]
[0.9252606  0.07473939]
[0.0301011 0.9698989]
[0.02414715 0.9758529 ]
[0.78414047 0.21585952]
[0.0301011 0.9698989]
[0.4261517  0.57384837]
[0.01743693 0.98256314]
[0.9209039  0.07909605]
[0.0540622 0.9459379]
[0.02414715 0.9758529 ]
[0.92735523 0.07264473]
[0.04284908 0.957151  ]
[0.9225497  0.07745028]
[0.03811106 0.9618889 ]
[0.14641725 0.85358274]
[0.9195102  0.08048978]
[0.928792 0.071208]
[0.4562491  0.54375094]
[0.02163504 0.978365  ]
[0.90894824 0.09105177]
[0.9194009  0.08059911]
[0.08521038 0.9147896 ]
[0

预测列表中的每个元素本身就是一个长度为2的列表。每个列表中两个值的总和为1。这是因为两列包含每种可能输出的概率：经历过的副作用和没有经历过的副作用。 预测列表中的每个元素都是所有可能输出上的概率分布。

第一列包含每个患者没有出现副作用的概率，用0表示。第二列包含每个患者发生副作用的概率，用1表示。

我们也可以只看最可能的预测。

In [ ]:
rouunded_predictions = np.argmax(predictions,axis=-1)
for i in rouunded_predictions:
  print(i)

1
1
1
0
1
0
1
0
1
1
1
1
0
1
0
0
1
0
1
0
0
1
1
0
1
1
1
0
1
1
0
1
0
1
1
0
0
1
1
0
0
1
0
1
1
1
1
1
0
0
0
1
0
0
1
1
1
0
0
0
1
0
1
1
0
0
1
1
0
0
0
0
1
1
0
1
0
0
0
1
0
1
1
0
0
0
1
1
1
0
0
1
1
0
1
1
0
1
1
0
1
1
0
1
0
1
1
0
0
1
1
0
1
1
0
1
1
1
1
0
0
0
1
0
0
1
0
1
1
0
0
1
1
1
0
0
0
0
0
1
0
1
0
1
1
1
1
1
0
0
0
0
0
1
1
0
1
0
0
1
0
0
1
1
0
0
0
0
1
0
1
0
0
0
1
1
1
1
1
0
1
0
0
1
0
1
1
0
1
0
0
1
1
0
1
0
1
1
0
1
1
1
0
1
0
0
1
1
0
1
0
1
0
1
0
0
0
1
1
1
1
0
0
1
0
1
1
1
0
1
1
1
1
0
1
0
1
1
0
0
1
0
1
0
0
0
1
0
1
1
1
1
0
0
1
1
1
1
0
0
0
0
0
0
1
0
1
1
0
1
0
1
0
1
1
1
0
1
0
1
0
1
1
1
1
1
1
1
0
0
1
0
1
0
0
0
0
0
0
1
1
1
1
1
1
1
1
0
0
0
0
0
1
0
1
0
1
0
1
0
0
0
1
1
1
0
1
1
1
0
0
0
1
0
1
1
1
1
0
1
1
1
1
1
0
0
1
0
1
1
1
1
1
0
0
0
0
0
0
0
1
0
0
0
1
0
1
0
0
1
1
1
0
1
0
1
0
1
0
1
0
1
1
1
1
0
0
1
1
0
0
1
0
0
0
0
0
1
1
1
1
1
0
1
0
0
0
0
1
0
1
0
1
0
1
1
0
1
1
0


从打印的预测结果中，我们可以观察模型中潜在的预测，但是，我们不能仅仅通过查看预测的输出来判断这些预测有多准确。

如果我们具有测试集的相应标签（在这种情况下，我们这样做），则可以将这些真实标签与预测标签进行比较，以判断模型评估的准确性。 在下一集中，我们将看到如何使用称为混淆矩阵的工具将其可视化。